## Ajuste de um perfil $\beta$ modificado

O perfil $\beta$ clássico não ajusta bem perfis com forte emissão no centro, como por exemplo, aglomerados de núcleo frio.
Nesse sentido Vikhlinin et al. 2006 propôs esse perfil para ajustar aglomerados com núcleo "cusp" e também propôs um perfil com uma queda mais ingreme para altos valores de $r$

\begin{equation}
    n_{p}n_{e} (r) = \frac{n_{0}^{2}}{(1+(r/r_{c})^{2})^{3\beta}} \frac{(r/r_{c})^{-\alpha}}{(1+(r/r_{c})^{2})^{-\alpha/2}} \frac{1}{(1+(r/r_{s})^{\gamma})^{\frac{\epsilon}{\gamma}}}
\end{equation}

Ou seja, esse novo perfil tem quatro parâmetros $(\alpha,\epsilon,\gamma,r_{s})$ adicionais em relação ao modelo $\beta$ clássico que tem apenas 3 $(n_{0},\beta,r_{c})$.

## Ajuste aos dados observacionais

A emissividade bremsstrahlung livre-livre é porporcional ao produto da densidade e a raíz quadrada da temperatura

\begin{equation}
     \epsilon^{ff} \propto n_{p}n_{e} \sqrt{T} \quad [erg \; s^{-1}{cm^{-3}}]
\end{equation}

Por outro lado, o que é extraído da observação é o brilho supercial projetado em raios-X ($SB^{\prime}$)  em unidades de contagens por segundos por unidade de área. 

\begin{equation}
SB(R) = A(R) \times SB^{\prime}(R)\\
    SB(R) = \int \epsilon^{ff} d\mathit{l} = \int^{\infty}_{R}  \epsilon^{ff} \frac{dr^{2}}{\sqrt{r^{2}-R^{2}}}
\end{equation}

Lembrando que $SB^{\prime}$ é obtido por anéis concêntricos. Para mais informações veja: http://cxc.harvard.edu/ciao/threads/radial_profile.

Onde $A(R)$ é o fator de conversão de taxa de contagens por unidade de área para fluxo. Sendo que $\epsilon^{ff}$ projetado na linha de visada é o brilho superficial.

\begin{equation}
    SB(R) = \int \epsilon^{ff} d\mathit{l} = \int  n_{p}n_{e} \sqrt{T}  d\mathit{l}
\end{equation}

Integrando em anéis infinitesimais onde a temperatura não vária consideravelmente, podemos escrever a equação acima como sendo:

\begin{equation}
    \frac{SB_{i}}{\sqrt{T_{i}}} = \int^{l_{i+1}}_{l_{i}}  n_{p}n_{e}   d\mathit{l}
\end{equation}

A equação acima que será utilizada para se obter o melhor ajuste. 

O fator de conversão é obtido ajustando um espectro para cada anel da imagem. Como o número de contagens é pequeno, fixa-se a temperatura e a metalicidade. Porém, a temperatura varia com raio, então assumimos o perfil de temperatura (Vikhlinin et al. 2006)

\begin{equation}
    T(r) = (1.11 T_{spec}) \times 1.35 \frac{(x/0.045)^{1.9}+0.45}{(x/0.045)^{1.9}+1} \frac{1}{1+(x/0.6)^{2}}
\end{equation}

Onde $x=r/r_{500}$ e $T_{spec}$ é a temperatura obtida pelo o ajuste espectral.

#### Extração do brilho superficial

A extração é feita partir de uma imagem corrigida por mapa de exposição no intervalo de energia entre 0.7 e 2 keV, sem as fontes pontuais, também é necessário uma imagem do background. Para realizar a extração é necessário definir anéis concêntricos. 

#### MCMC

In [70]:
from scipy.integrate import qaud
import numpy as np
from sherpa.astro.ui import *
from sherpa.models import*
from beta1d_mod import*
import emcee

In [54]:
def integral(pars,r):
    (r,rc,rs,a,b,e,g,n0) = pars
    old_settings = np.seterr(all='warn')  #seterr to known value
    beta=(n0**2)*((r/rc)**(-a))/( (1+(r/rc)**(2))**(3*b-a/2) )
    res1 = beta/( 1+(r/rs)**(g) )**(e/g)
    res = 2*res1*r/(r*r-R*R)**(1/2)
    return res

def Eprojected(pars,x):
    (rc,rs,a,b,e,g,n0) = pars
    n = len(x)
    out = []
    for i in range(n):
        res = x[i]
        print(rc,rs,a,b,e,g,n0,res)
        aux=quad(integral,x,1e6,args=(rc,rs,a,b,e,g,n0,res))
        out.append(aux[0])
    return np.array(out)


In [112]:
# Definindo funções
def betamod(r,rc,rs,a,b,e,n0):
    g=3
    old_settings = np.seterr(all='warn')  #seterr to known value
    beta=(n0**2)*((r/rc)**(-a))/( (1+(r/rc)**(2))**(3*b-a/2) )
    res = beta/( 1+(r/rs)**(g) )**(e/g)
    return res

def integral(r,R,rc,rs,a,b,e,n0):
    old_settings = np.seterr(all='warn')  #seterr to known value
    res = 2*betamod(r,rc,rs,a,b,e,n0)*r/(r*r-R*R)**(1/2)
    return res

def Eprojected(pars,x):
    out = []
    for i in range(len(x)):
        R = x[i]
        aux=quad(integral,R,np.inf,args=(R,pars[0],pars[1],pars[2],pars[3],pars[4],pars[5]),epsabs=1e-10)
        res=aux[0]
        out.append(res)
    return np.array([out])

SBV = np.vectorize(Eprojected, otypes=[np.float])

In [113]:
par0 = 80,400,0.5,1.0,1.0,1
para = np.array(par0)
Eprojected(para,x)

array([[  4.62867006e+01,   4.22800957e+01,   3.84248945e+01,
          3.47395560e+01,   3.12403596e+01,   2.79409785e+01,
          2.48521380e+01,   2.19813765e+01,   1.93329241e+01,
          1.69077044e+01,   1.47034576e+01,   1.27149749e+01,
          1.09344264e+01,   9.35176094e+00,   7.95515244e+00,
          6.73146503e+00,   5.66671396e+00,   4.74649939e+00,
          3.95639664e+00,   3.28229058e+00,   2.71064775e+00,
          2.22872439e+00,   1.82471242e+00,   1.48782893e+00,
          1.20835654e+00,   9.77642890e-01,   7.88068380e-01,
          6.32990240e-01,   5.06670691e-01,   4.04195632e-01,
          3.21389196e-01,   2.54728311e-01,   2.01260314e-01,
          1.58525713e-01,   1.24487402e-01,   9.74669784e-02,
          7.60883515e-02,   5.92284719e-02,   4.59747782e-02,
          3.55888121e-02,   2.74753778e-02,   2.11566025e-02,
          1.62502626e-02,   1.24517829e-02,   9.51936459e-03,
          7.26175759e-03,   5.52825190e-03,   4.20052027e-03,
        

In [55]:
par0 = [80,400,0.5,1.0,1.0,3.0,0.0210105]
Eprojected(par0,x)

80 400 0.5 1.0 1.0 3.0 0.0210105 53.8125


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# del(rc,rs,a,b,e,g,n0)

In [ ]:
# Maximum likelihood estimation
# Chi-square function
def chi2(theta,x,y,yerr):
    rc,rs,a,b,g,n0 = theta
    model = SBV(rc,rs,a,b,g,n0,x)
    inv_sigma2 = 1.0/(yerr**2)
    return np.sum((model-y)**2*inv_sigma2)

# Log likelihood function
def lnlike(theta,x,y,yerr):
    return -0.5*(chi2(theta,x,y,yerr))

In [ ]:
def lnprior(theta):
    rc,rs,a,b,e,n0 = theta
    if 0.0 < a and 0.0 < b and 0.0 < e < 5.0 and 0.0 < rs and 0.0 < rc and 0.0 < n0:
        return 0.0
    return -np.inf

def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

In [31]:
from astropy.io import fits
import astropy.io.ascii as at
# fits_image_filename = fits.util.get_testdata_filepath('00524_rprofile.fits ')

conv = at.read("cnt_rate")
hdul = fits.open("00524_rprofile.fits")

data = hdul[1].data
cols = hdul[1].columns
Rbin = (data['R'][:,0]+data['R'][:,1])/2
SBC = data['SUR_BRI']*1e8
SBC_ERR = data['SUR_BRI_ERR']*1e8
NC = data['NET_COUNTS']

In [32]:
x, y, yerr = Rbin, SBC, SBC_ERR
par0 = 80,400,0.5,1.0,1.0,3,0.0210105
par0a = np.array(par0)
ndim, nwalkers = 7, 200
pos = [par0+1e-1*par0a*np.random.randn(ndim) for i in range(nwalkers)]

In [ ]:
Eprojected([par0],100)

In [ ]:
# implementar no sherpa
import scipy.optimize as op
nll = lambda *args: -lnprob(*args)
result = op.minimize(nll, [par0], args=(x, y, yerr))
rc_l,rs_l,a_l,b_l,e_l,n0_l = result["x"]

In [ ]:
print(rc_l,rs_l,a_l,b_l,e_l,n0_l)

In [ ]:
import time
t = time.time()

sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlike, args=(x,y,yerr))
sampler.run_mcmc(pos, 200)

print(str(time.time()-t))

In [ ]:
import corner
samples = sampler.chain[:, 50:, :].reshape((-1, ndim))
fig = corner.corner(samples)

In [ ]:
hdr = hdul[0].header
# list(hdr.keys())

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(Rbin,SBC)
plt.xscale("log")
plt.yscale("log")
plt.ylim(1e-5,0.12)
plt.show()

In [ ]:
plt.scatter(Rbin,NC)
plt.xscale("log")
plt.yscale("log")
plt.show()

#### 1 - Preparando a imagem

In [ ]:
import astropy.io.ascii as at
# from ciao_contrib.runtool import *

#--- Carregando tabelas
clt = at.read("cluster.txt")
obsid="00"+str(clt["obsid"][0])
pixscale=clt["pixscale"][0]

# arquivo de eventos filtrado por flares
evtg=obsid+"_gti_evt2.fits"  
evtge="excl_gti_evt2.fits"

# Excluindo fontes pontuais
# dmcopy(evtg+"[exclude sky=region(ps.reg)] ",evtge) 

# Definindo intervalo de energia
# dmcopy(evtg+"[energy=700:2000]","0.7-2.0"+evtge) 

#### 2 - Definindo Anéis Concêntricos

In [ ]:
import numpy as np

t500=360
ri, rfim = 0.1*t500, 2*t500, 
naneis = int( np.log((rfim/ri) -1)/np.log(1.05))

rbin = [50*(1.05)**(i+1) for i in range(naneis)]

saida = "aneis.reg"
with open(saida,"w") as file:
    for i in range(naneis-1):
        file.write("annulus("+str(clt["xcen"][0])+","+str(clt["ycen"][0])+","+str(rbin[i])+","+str(rbin[i+1])+") \n")

rbkg=rfim+50
with open("bkg_"+saida,"w") as file:
        file.write("annulus("+str(clt["xcen"][0])+","+str(clt["ycen"][0])+","+str(rfim)+","+str(rbkg)+")\n")


In [ ]:
dmextract.punlearn()
dmextract.infile = evtge+"[bin sky=@"+saida+"]"
dmextract.outfile = obsid+"_rprofile.fits"
dmextract.bkg = evtge+"[bin sky=@bkg_"+saida+"]"
dmextract.opt = "generic"
dmextract()

dmtcalc.punlearn()
dmtcalc.infile= obsid+"_rprofile.fits"
dmtcalc.outfile=obsid+"_rprofile_rmid.fits"
dmtcalc.expression="rmid=0.5*(R[0]+R[1])"
dmtcalc()


In [ ]:
para[0]

In [26]:
# load_data(1,"...")
# frzpar=5*[False]+[True,False]
# load_user_model(Eprojected, "mybeta")
# add_user_pars("mybeta", ["rc","rs","alpha","beta","epsilon","gamma","n0"], par0,parfrozen=frzpar)
# plot_data()